In [88]:
import torch
import numpy as np
from math import sin, cos
from typing import List, Dict

SENTENCE = "<s> I LIKE PIE VERY MUCH PIE ."
SENTENCE2 = "<s> WE ARE YOUNG ."
DIM_EMB = 512
torch.manual_seed(7)

vocab = dict()
i = 0
for token in SENTENCE.split():
    if token not in vocab:
        vocab[token] = i
        i += 1

for token in SENTENCE2.split():
    if token not in vocab:
        vocab[token] = i
        i += 1
        
vocab["<unk>"] = i
vocab["Peter"] = i + 1

print(vocab)

inv_vocab = dict()
for word in vocab:
    inv_vocab[vocab[word]] = word

DICT_SIZE = len(vocab)

def getBatch(sents: List[str], vocab: Dict[str, int]) -> torch.Tensor:
    #Get max sentence length
    DICT_SIZE = len(vocab)
    maxLen = 0
    for sent in sents:
        maxLen = max(maxLen, len(sent.split()))
    oneHotBatch = np.zeros((len(sents), maxLen, DICT_SIZE), dtype=float)
    for i in range(len(sents)):
        curr_sent = sents[i].split()
        for j in range(len(curr_sent)):
            word = curr_sent[j]
            oneHotBatch[i][j][vocab[word]] = 1
    return torch.as_tensor(oneHotBatch, dtype=torch.float32)

batch_tst = getBatch([SENTENCE, SENTENCE], vocab)

oneHotSent = np.zeros((len(SENTENCE.split()), DICT_SIZE), dtype=float)

for i in range(len(SENTENCE.split())):
    word = SENTENCE.split()[i]
    oneHotSent[i][vocab[word]] = 1

oneHotTorch = torch.as_tensor(oneHotSent, dtype=torch.float32)
    
class FUCKYOU(torch.nn.Module):
    def __init__(self, vocab_size, batch_size):
        super().__init__()
        self.DICT_SIZE = vocab_size
        self.batch_size = batch_size #@TODO CHANGE
        self.transformerHeadConcat = 64 #@TODO CHANGE
        
        Wq_skeleton = torch.tensor(np.zeros((DIM_EMB, 64), dtype=float), dtype=torch.float32,).normal_(0, 1)
        self.Wq = torch.nn.Parameter(Wq_skeleton, requires_grad=True)

        Wk_skeleton = torch.tensor(np.zeros((DIM_EMB, 64), dtype=float), dtype=torch.float32,).normal_(0, 1)
        self.Wk = torch.nn.Parameter(Wk_skeleton, requires_grad=True)

        Wv_skeleton = torch.tensor(np.zeros((DIM_EMB, 64), dtype=float), dtype=torch.float32,).normal_(0, 1)
        self.Wv = torch.nn.Parameter(Wv_skeleton, requires_grad=True)
        
        Wo_skeleton = torch.tensor(np.zeros((self.transformerHeadConcat, DIM_EMB), dtype=float), dtype=torch.float32,).normal_(0, 1)
        self.Wo = torch.nn.Parameter(Wo_skeleton, requires_grad=True)
        
        #Embeddings!
        We_skeleton = torch.tensor(np.zeros((DICT_SIZE, DIM_EMB), dtype=float), dtype=torch.float32,).normal_(0, 1)
        self.We = torch.nn.Parameter(We_skeleton, requires_grad=True)
        
        Wff_skeleton = torch.tensor(np.zeros((DIM_EMB, DIM_EMB), dtype=float), dtype=torch.float32,).normal_(0, 1)
        self.Wff = torch.nn.Parameter(Wff_skeleton, requires_grad=True)
        Bff_skeleton = torch.tensor(np.zeros((DIM_EMB), dtype=float), dtype=torch.float32,).normal_(0, 1)
        self.Bff = torch.nn.Parameter(Bff_skeleton, requires_grad=True)
        
        Wproj_skeleton = torch.tensor(np.zeros((DIM_EMB, DICT_SIZE), dtype=float), dtype=torch.float32,).normal_(0, 1)
        self.Wproj = torch.nn.Parameter(Wproj_skeleton, requires_grad=True)
        Bproj_skeleton = torch.tensor(np.zeros((DICT_SIZE), dtype=float), dtype=torch.float32,).normal_(0, 1)
        self.Bproj = torch.nn.Parameter(Bproj_skeleton, requires_grad=True)
        
        torch.nn.utils.clip_grad_norm_(self.parameters(), 5)
        self.optimizer = torch.optim.Adam(self.parameters(), lr = 0.001)
        
    
    #DATA PREPARATION DONE

    # Create self attention
    # also need to add a mask here
    # @TODO Add a Bias to all Wsz
    def add_transformer_head_with_mask(self, DIM_EMB, Batch, mask):
        Q = Batch.matmul(self.Wq)
        V = Batch.matmul(self.Wv)
        K = Batch.matmul(self.Wk)

        tmp1 = Q.matmul(K.permute(0, 2, 1))
        #tmp1 = torch.t(K).matmul(Q)

        tmp = tmp1/(64**(1/2))
        #@TODO this might be wrong for dim=-1
        softM = torch.nn.Softmax(dim=-1)
        #apply maks here
        tmp = tmp.masked_fill(mask == 0, -1e9)

        a = softM(tmp)

        z = a.matmul(V)
        return z

    # @TODO add a bias to Wo
    def addLinearProj(self, concatTransHead, DIM_EMB):
        '''Already concatenataed all heads'''

        return concatTransHead.matmul(self.Wo)

    #Normalize
    def layerNorm(self, mat):
        return (mat - mat.mean(dim=1, keepdim=True)) / (mat.std(dim=1, keepdim=True))

    EPOCHS=10
    def fwd(self, oneHotTorch: torch.Tensor):
        
        #Positional embeddings here
        Batch = oneHotTorch.matmul(self.We)
        Pos_emb = np.zeros((Batch.shape[1], Batch.shape[2]), dtype=float)
        for p in range(len(SENTENCE.split())):
            for i in range(DIM_EMB//2):
                val = p/(10000**(2*i/DIM_EMB))
                Pos_emb[p][2*i] = sin(val)
                Pos_emb[p][2*i + 1] = cos(val)

        Pos_emb_var = torch.nn.Parameter(torch.tensor(Pos_emb, dtype=torch.float32), requires_grad=False)

        Batch = Batch*(DIM_EMB**(1/2))
        
        #print(Batch)
        Batch = Batch + Pos_emb_var
        #Mask the batch post pos_emb
        mask1 = torch.sum(oneHotTorch, dim=2) > 0
        mask1 = mask1.reshape(2,8,1).float()
        Batch = Batch*mask1
        #print(Batch)

        #Create MASK HERE
        #
        mask = np.triu(np.ones((len(SENTENCE.split()), len(SENTENCE.split()))), k=1)
        mask = torch.from_numpy(mask) == 0

        transformer_head = self.add_transformer_head_with_mask(DIM_EMB, Batch, mask)
        postAttention = self.addLinearProj(transformer_head, DIM_EMB)

        # Add residual connection to the embeddings
        postAttention = postAttention + Batch
        #print(postAttention)

        normalizedPostAttention = self.layerNorm(postAttention)
        #print(normalizedPostAttention)
        # @TODO no dropout
        # Take the output of the attention and do a one layer FF that outputs size of
        activation = torch.nn.ReLU()
        postFF = activation(normalizedPostAttention.matmul(self.Wff) + self.Bff)

        #Residual
        preSoftmaxProj = self.layerNorm(postFF + normalizedPostAttention)
        #print(preSoftmaxProj)
        #Projexct

        linearLay = activation(preSoftmaxProj.matmul(self.Wproj) + self.Bproj)
        #Softmax
        #print(linearLay)
        #softmax = torch.nn.LogSoftmax(dim=1)
        #output = softmax(linearLay)
        #print(output)
        #return output
        return linearLay*mask1 #Not sure if this is correct

    #@TODO
    def getOneHot(self, truth):
        ret = torch.Tensor(self.batch_size).long() 

model = FUCKYOU(DICT_SIZE, len(SENTENCE.split()))
EPOCHS=100

def printSent(softmaxd, vocab):
    words = [[]]
    for j in range(len(softmaxd)):
        words.append([])
        for i in range(len(softmaxd[j])):
            _, idx = torch.max(softmaxd[j][i], 0)
            words[j].append(vocab[int(idx)])
    for arr in words:
        print(" ".join(arr))
    
def getTruth(oneHotSent):
    truth = []
    for j in range(len(oneHotSent)):
        truth.append([])
        for i in range(len(oneHotSent[j])):
            _, idx = torch.max(oneHotSent[j][i], 0)
            truth[j].append(idx)
    return torch.Tensor(truth).long()

for e in range(EPOCHS):
    model.optimizer.zero_grad()
    fwd = model.fwd(batch_tst)
    printSent(fwd, inv_vocab)
    #print(batch_tst)
    truth = getTruth(batch_tst)
    #print(truth)
    print("fwd", fwd.shape)
    print("truth", truth.shape)
    loss = torch.nn.functional.cross_entropy(fwd.permute(0, 2, 1), truth, size_average = True, ignore_index=11)
    print("Loss: ", loss.item())
    loss.backward()
    model.optimizer.step()

    



{'<s>': 0, 'I': 1, 'LIKE': 2, 'PIE': 3, 'VERY': 4, 'MUCH': 5, '.': 6, 'WE': 7, 'ARE': 8, 'YOUNG': 9, '<unk>': 10, 'Peter': 11}
<unk> I Peter ARE ARE <unk> LIKE <unk>
<unk> I Peter ARE ARE <unk> LIKE <unk>

tensor([[0, 1, 2, 3, 4, 5, 3, 6],
        [0, 1, 2, 3, 4, 5, 3, 6]])
fwd torch.Size([2, 8, 12])
truth torch.Size([2, 8])
Loss:  28.003015518188477
. I YOUNG I ARE . ARE .
. I YOUNG I ARE . ARE .

tensor([[0, 1, 2, 3, 4, 5, 3, 6],
        [0, 1, 2, 3, 4, 5, 3, 6]])
fwd torch.Size([2, 8, 12])
truth torch.Size([2, 8])
Loss:  22.425607681274414
. I YOUNG I Peter . ARE .
. I YOUNG I Peter . ARE .

tensor([[0, 1, 2, 3, 4, 5, 3, 6],
        [0, 1, 2, 3, 4, 5, 3, 6]])
fwd torch.Size([2, 8, 12])
truth torch.Size([2, 8])
Loss:  17.448501586914062
. I ARE I VERY . I .
. I ARE I VERY . I .

tensor([[0, 1, 2, 3, 4, 5, 3, 6],
        [0, 1, 2, 3, 4, 5, 3, 6]])
fwd torch.Size([2, 8, 12])
truth torch.Size([2, 8])
Loss:  12.648316383361816
. I ARE ARE VERY . PIE .
. I ARE ARE VERY . PIE .

tensor([[0

. I Peter I VERY . PIE .
. I Peter I VERY . PIE .

tensor([[0, 1, 2, 3, 4, 5, 3, 6],
        [0, 1, 2, 3, 4, 5, 3, 6]])
fwd torch.Size([2, 8, 12])
truth torch.Size([2, 8])
Loss:  1.483670711517334
. I Peter I VERY . PIE .
. I Peter I VERY . PIE .

tensor([[0, 1, 2, 3, 4, 5, 3, 6],
        [0, 1, 2, 3, 4, 5, 3, 6]])
fwd torch.Size([2, 8, 12])
truth torch.Size([2, 8])
Loss:  1.4857368469238281
. I Peter I VERY . PIE .
. I Peter I VERY . PIE .

tensor([[0, 1, 2, 3, 4, 5, 3, 6],
        [0, 1, 2, 3, 4, 5, 3, 6]])
fwd torch.Size([2, 8, 12])
truth torch.Size([2, 8])
Loss:  1.4832369089126587
. I Peter I VERY . PIE .
. I Peter I VERY . PIE .

tensor([[0, 1, 2, 3, 4, 5, 3, 6],
        [0, 1, 2, 3, 4, 5, 3, 6]])
fwd torch.Size([2, 8, 12])
truth torch.Size([2, 8])
Loss:  1.476409673690796
. I Peter I VERY . PIE .
. I Peter I VERY . PIE .

tensor([[0, 1, 2, 3, 4, 5, 3, 6],
        [0, 1, 2, 3, 4, 5, 3, 6]])
fwd torch.Size([2, 8, 12])
truth torch.Size([2, 8])
Loss:  1.4772595167160034
. I Peter I 

. I Peter I VERY . PIE .
. I Peter I VERY . PIE .

tensor([[0, 1, 2, 3, 4, 5, 3, 6],
        [0, 1, 2, 3, 4, 5, 3, 6]])
fwd torch.Size([2, 8, 12])
truth torch.Size([2, 8])
Loss:  1.416321039199829
. I Peter I VERY . PIE .
. I Peter I VERY . PIE .

tensor([[0, 1, 2, 3, 4, 5, 3, 6],
        [0, 1, 2, 3, 4, 5, 3, 6]])
fwd torch.Size([2, 8, 12])
truth torch.Size([2, 8])
Loss:  1.4145995378494263
. I Peter I VERY . PIE .
. I Peter I VERY . PIE .

tensor([[0, 1, 2, 3, 4, 5, 3, 6],
        [0, 1, 2, 3, 4, 5, 3, 6]])
fwd torch.Size([2, 8, 12])
truth torch.Size([2, 8])
Loss:  1.4125902652740479
. I Peter I VERY . PIE .
. I Peter I VERY . PIE .

tensor([[0, 1, 2, 3, 4, 5, 3, 6],
        [0, 1, 2, 3, 4, 5, 3, 6]])
fwd torch.Size([2, 8, 12])
truth torch.Size([2, 8])
Loss:  1.4109009504318237
. I Peter I VERY . PIE .
. I Peter I VERY . PIE .

tensor([[0, 1, 2, 3, 4, 5, 3, 6],
        [0, 1, 2, 3, 4, 5, 3, 6]])
fwd torch.Size([2, 8, 12])
truth torch.Size([2, 8])
Loss:  1.4083589315414429
. I Peter I